In [1]:
import pickle
import numpy as np
from dataset import Dataset
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

In [2]:
with open("sensor_data_filtered.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

dataset = Dataset(sensor_data, labels, interp_funcs)

In [3]:
results = {
    "day1": {
        "mat1": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []},
        "mat2": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []}
    },
    "day2": {
        "mat1": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []},
        "mat2": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []}
    },
    "day3": {
        "mat1": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []},
        "mat2": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []}
    }
}

for day in range(1, 4):
    for matrix in range(1, 3):
        for s in range(0, 8, 2):
            s_l = s
            s_r = s+1
            print(f"Day {day}, Mat {matrix}, Sensors {s_l} & {s_r}")

            X, y, _ = dataset.get_sensor_pair_cls(day, matrix,
                                                  (s_l, s_r),
                                                  num_samples=100,
                                                  as_log=True,
                                                  as_mean=False)

            scaler = MinMaxScaler()
            X = scaler.fit_transform(X.T).T

            clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "Decision Tree", "accuracy": np.round(accuracy, 3)}
            )

            clf = GaussianNB()
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "GaussianNB", "accuracy": np.round(accuracy, 3)}
            )

            extra_tree = ExtraTreeClassifier(random_state=0)
            clf = BaggingClassifier(extra_tree, random_state=0)
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "BaggingTree", "accuracy": np.round(accuracy, 3)}
            )

            clf = SVC(kernel="linear")
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "SVM", "accuracy": np.round(accuracy, 3)}
            )

            clf = XGBClassifier(objective="multi:softprob")
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "XGB", "accuracy": np.round(accuracy, 3)}
            )

            clf = LogisticRegression(max_iter=5000)
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "Logistic Reg.", "accuracy": np.round(accuracy, 3)}
            )

            clf = MLPClassifier(max_iter=5000)
            accuracy = np.mean(cross_val_score(clf, X, y, cv=5))
            results[f"day{day}"][f"mat{matrix}"][f"s_{s_l}_{s_r}"].append(
                {"clf": "MLP", "accuracy": np.round(accuracy, 3)}
            )

Day 1, Mat 1, Sensors 0 & 1
Day 1, Mat 1, Sensors 2 & 3
Day 1, Mat 1, Sensors 4 & 5
Day 1, Mat 1, Sensors 6 & 7
Day 1, Mat 2, Sensors 0 & 1
Day 1, Mat 2, Sensors 2 & 3
Day 1, Mat 2, Sensors 4 & 5
Day 1, Mat 2, Sensors 6 & 7
Day 2, Mat 1, Sensors 0 & 1
Day 2, Mat 1, Sensors 2 & 3
Day 2, Mat 1, Sensors 4 & 5
Day 2, Mat 1, Sensors 6 & 7
Day 2, Mat 2, Sensors 0 & 1
Day 2, Mat 2, Sensors 2 & 3
Day 2, Mat 2, Sensors 4 & 5
Day 2, Mat 2, Sensors 6 & 7
Day 3, Mat 1, Sensors 0 & 1
Day 3, Mat 1, Sensors 2 & 3
Day 3, Mat 1, Sensors 4 & 5
Day 3, Mat 1, Sensors 6 & 7
Day 3, Mat 2, Sensors 0 & 1
Day 3, Mat 2, Sensors 2 & 3
Day 3, Mat 2, Sensors 4 & 5
Day 3, Mat 2, Sensors 6 & 7


In [4]:
results

{'day1': {'mat1': {'s_0_1': [{'clf': 'Decision Tree', 'accuracy': 0.795},
    {'clf': 'GaussianNB', 'accuracy': 0.792},
    {'clf': 'BaggingTree', 'accuracy': 0.885},
    {'clf': 'SVM', 'accuracy': 0.69},
    {'clf': 'XGB', 'accuracy': 0.847},
    {'clf': 'Logistic Reg.', 'accuracy': 0.638},
    {'clf': 'MLP', 'accuracy': 0.702}],
   's_2_3': [{'clf': 'Decision Tree', 'accuracy': 0.935},
    {'clf': 'GaussianNB', 'accuracy': 0.952},
    {'clf': 'BaggingTree', 'accuracy': 0.95},
    {'clf': 'SVM', 'accuracy': 0.893},
    {'clf': 'XGB', 'accuracy': 0.87},
    {'clf': 'Logistic Reg.', 'accuracy': 0.94},
    {'clf': 'MLP', 'accuracy': 0.948}],
   's_4_5': [{'clf': 'Decision Tree', 'accuracy': 0.92},
    {'clf': 'GaussianNB', 'accuracy': 0.975},
    {'clf': 'BaggingTree', 'accuracy': 0.938},
    {'clf': 'SVM', 'accuracy': 0.888},
    {'clf': 'XGB', 'accuracy': 0.91},
    {'clf': 'Logistic Reg.', 'accuracy': 0.805},
    {'clf': 'MLP', 'accuracy': 0.928}],
   's_6_7': [{'clf': 'Decision Tree'